In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests

# 표를 불러오기 위한 라이브러리
from html_table_parser import parser_functions as parser
import collections

collections.Callable = collections.abc.Callable
# 403 에러 방지
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [ ]:
# csv 파일 읽기

file_name = "반팔" + "_url_list.csv"
data = pd.read_csv(file_name, names=["name", "url"])
urls = data["url"].values
print(urls.shape, urls)

In [ ]:
url_df = pd.DataFrame(data, columns=["name", "url"])
url_list = url_df.values.tolist()

print(url_list)

In [8]:
url = url_list[1][1]

page = requests.get(url, headers=headers)
soup = bs(page.text, "lxml")
table = soup.find_all(attrs={"class": "table_th_grey"})

# print(table)

p = parser.make2d(table[0])
size_df = pd.DataFrame(p[3:], columns=p[0])
print(size_df)

   cm  총장  어깨너비  가슴단면 소매길이
0  XS  58    44    50   19
1   S  67    48    52   22
2   M  70  50.5  55.5   24
3   L  72    52    58   25
4  XL  74  53.5  60.5   26


In [9]:
review_list = []
page = requests.get(url, headers=headers)
soup = bs(page.text, "lxml")

i = 0

while i >= 0:
    try:
        review = soup.find_all(attrs={"class": "size_content"})[i].get_text()
        review_list.append(review.split())
        i += 1
    except:
        break

In [17]:
for i in range(10):
    print(review_list[i])

['[회원추천]', '(남자', '170cm/63kg)', '기준', '적당함', 'M', 'Size', '구매']
['[회원추천]', '(남자', '168cm/56kg)', '기준', '적당함', 'M', 'Size', '구매']
['[회원추천]', '(여자', '169cm/53kg)', '기준', '적당함', 'M', 'Size', '구매']
['[회원추천]', '(남자', '173cm/63kg)', '기준', '적당함', 'L', 'Size', '구매']
['[회원추천]', '(여자', '160cm/45kg)', '기준', '적당함', 'S', 'Size', '구매']
['[회원추천]', '(여자', '158cm/44kg)', '기준', '적당함', 'S', 'Size', '구매']
['[회원추천]', '(여자', '170cm/75kg)', '기준', '큼', 'L', 'Size', '구매']
['[회원추천]', '(여자', '165cm/55kg)', '기준', '적당함', 'S', 'Size', '구매']
['[회원추천]', '(여자', '170cm/63kg)', '기준', '큼', 'XL', 'Size', '구매']
['[회원추천]', '(여자', '158cm/50kg)', '기준', '적당함', 'S', 'Size', '구매']


In [ ]:
review_list2 = []
for rv in review_list:
    temp = []
    for i in range(len(rv)):
        if i == 1:
            temp.append(rv[i][1:])
        elif i == 2:
            temp.append(rv[i].split("/")[0])
            temp.append(rv[i].split("/")[1][:-1])
        elif i == 0 or i == 3 or i == 6 or i == 7:
            continue
        else:
            temp.append(rv[i])
        review_list2.append(temp)

print(review_list2)

In [13]:
col_name = ["성별", "키", "몸무게", "후기", "사이즈"]
review_df = pd.DataFrame(review_list2, columns=col_name)

print(review_df[:10])
print(review_df[-10:])

   성별      키   몸무게   후기 사이즈
0  남자  170cm  63kg  적당함   M
1  남자  170cm  63kg  적당함   M
2  남자  170cm  63kg  적당함   M
3  남자  170cm  63kg  적당함   M
4  남자  168cm  56kg  적당함   M
5  남자  168cm  56kg  적당함   M
6  남자  168cm  56kg  적당함   M
7  남자  168cm  56kg  적당함   M
8  여자  169cm  53kg  적당함   M
9  여자  169cm  53kg  적당함   M
     성별      키   몸무게   후기 사이즈
390  여자  166cm  50kg  적당함   S
391  여자  166cm  50kg  적당함   S
392  여자  168cm  49kg  적당함   M
393  여자  168cm  49kg  적당함   M
394  여자  168cm  49kg  적당함   M
395  여자  168cm  49kg  적당함   M
396  남자  176cm  60kg  적당함   M
397  남자  176cm  60kg  적당함   M
398  남자  176cm  60kg  적당함   M
399  남자  176cm  60kg  적당함   M


In [25]:
size_review_df = pd.merge(left = size_df , right = review_df, how = "inner", left_on = "cm", right_on="사이즈")
size_review_df.sort_values(['사이즈', '후기'], ascending = [True, True])
print(size_review_df)

     cm  총장  어깨너비  가슴단면 소매길이  성별      키   몸무게   후기 사이즈
0    XS  58    44    50   19  여자  163cm  50kg  적당함  XS
1    XS  58    44    50   19  여자  163cm  50kg  적당함  XS
2    XS  58    44    50   19  여자  163cm  50kg  적당함  XS
3    XS  58    44    50   19  여자  163cm  50kg  적당함  XS
4     S  67    48    52   22  여자  160cm  45kg  적당함   S
..   ..  ..   ...   ...  ...  ..    ...   ...  ...  ..
395  XL  74  53.5  60.5   26  남자  178cm  70kg  적당함  XL
396  XL  74  53.5  60.5   26  남자  168cm  70kg  적당함  XL
397  XL  74  53.5  60.5   26  남자  168cm  70kg  적당함  XL
398  XL  74  53.5  60.5   26  남자  168cm  70kg  적당함  XL
399  XL  74  53.5  60.5   26  남자  168cm  70kg  적당함  XL

[400 rows x 10 columns]


In [28]:
#join한 dataframe csv파일로 저장
file_name = "반팔티사이즈리뷰"
size_review_df.to_csv(file_name)